In [6]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from IPython.display import display
import os

In [ ]:
#CARREGANDO A BASE DE DADOS

data = pd.read_excel('data/base_dados_modelo.xlsx')

cols_select = ['date', 'selic', 'acum_12_meses', 'meta_inflacao', 'meta_modificada',
               'JUROS_NATURAL', 'REER', 'IBC_BR', 'COMMODITIES', 'EXP_PIB',
               'EXP_INF', 'gap_inflacao', 'LOSS_12']

#CRIANDO A FUNÇÃO PARA FAZER A TABELA  DE RESULTADO DO TESTE DE ESTACIONARIDADE

def adf_test_results(data):
    cols_select = ['date', 'selic', 'acum_12_meses', 'meta_inflacao', 'meta_modificada',
                   'JUROS_NATURAL', 'REER', 'IBC_BR', 'COMMODITIES', 'EXP_PIB',
                   'EXP_INF', 'gap_inflacao', 'LOSS_12']

    data = data[cols_select].dropna()

    
    data['DATE'] = pd.to_datetime(data['date'])

    # Criando a variável de expectativa do PIB
    data_model = data.copy()
    data_model['EXP_GAP_INF'] = data_model['EXP_INF'] - data_model['meta_modificada']
    data_model = data_model.dropna()

    columns_to_test = ["selic", "JUROS_NATURAL", "gap_inflacao", "EXP_GAP_INF",
                       "EXP_PIB", "LOSS_12"]
    
    dict_replace = {"selic":"Selic", 
                                "JUROS_NATURAL":"Tx. Juros Natural",
                                "EXP_GAP_INF": "Gap de Inf Esperada",
                                "EXP_PIB":"Expectativa PIB",
                                "LOSS_12":"Perda de Credibilidade"}
    
    ts_data_sub = data_model.set_index('date')[columns_to_test]

    ts_data_sub.rename(columns=dict_replace, inplace=True)

    def run_adf_test(series, variable_name):
        result = adfuller(series, maxlag=1, regression='n')
        return {
            'Variável': variable_name,
            'Estatística de Teste': round(result[0], 3),
            'Significância': '***' if result[1] < 0.001 else ('**' if result[1] < 0.01 else ('*' if result[1] < 0.05 else ''))
        }

    results_list = [run_adf_test(ts_data_sub[var], var) for var in dict_replace.values()]

    
    df_resultados = pd.DataFrame(results_list)
    df_resultados = df_resultados[['Variável', 'Estatística de Teste', 'Significância']]

    display(df_resultados)

    notes = [
        "Notas: A tabela mostra os resultados do teste ADF (Augmented Dickey-Fuller).",
        "Estatísticas de teste e níveis de significância são reportados.",
        "* p < 0.05, ** p < 0.01, *** p < 0.001"
    ]
    for note in notes:
        print(note)

# Teste de estacionaridade

Esta tabela apresenta os resultados para o teste ADF (Augmented Dick-Fuller), todas as variávies, exceto a expectativa do PIB são estacionárias, portanto os modelos serão estimados em nível.

In [ ]:
adf_test_results(data)

,Variável,Estatística de Teste,Significância
0,Selic,-2.567,**
1,Tx. Juros Natural,-11.642,***
2,Gap de Inf Esperada,-4.102,***
3,Expectativa PIB,-0.941,
4,Perda de Credibilidade,-8.090,***


Notas: A tabela mostra os resultados do teste ADF (Augmented Dickey-Fuller).
Estatísticas de teste e níveis de significância são reportados.
* p < 0.05, ** p < 0.01, *** p < 0.001
